# 1. Packages

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
torch.set_default_tensor_type(torch.DoubleTensor)
import numpy as np
import pandas as pd


import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

import datetime
from datetime import timedelta,date,datetime
import copy
from scipy import stats
import math
import collections
import importlib

import sklearn
from sklearn import manifold
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler,RobustScaler,OneHotEncoder
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.mixture import GaussianMixture

import networkx as nx
import community
import dgl

import matplotlib.pyplot as plt 
import seaborn as sns

import os
import argparse
import time
import random

plt.rcParams['font.sans-serif'] = ['SimHei']  # font family set
plt.rcParams['axes.unicode_minus'] = False  # for minus display
plt.style.use('seaborn')
sns.set_style(style='ticks') #set background
plt.rcParams['figure.dpi'] = 200
%config InlineBackend.figure_format = 'svg' #show svg pictures
import warnings
warnings.filterwarnings("ignore")

# 2.Dataset

In [ ]:
samples_2 = pd.read_csv('./data_gnn/samples_2.csv')
samples_2 = samples_2.drop(columns=['level1_x_mean', 'level2_x_mean',
       'level3_x_mean', 'level4_x_mean', 'score1_x_mean', 'score2_x_mean',
       'exp1_x_mean', 'exp2_x_mean', 'exp3_x_mean', 'online_time_x_mean',
       'level1_y_mean', 'level2_y_mean', 'level3_y_mean', 'level4_y_mean',
       'score1_y_mean', 'score2_y_mean', 'exp1_y_mean', 'exp2_y_mean',
       'exp3_y_mean', 'online_time_y_mean'],inplace=True)

In [ ]:
train = samples_2.iloc[:,2:-2]
target = samples_2.iloc[:,-2]
scaler = RobustScaler() #normalize for robust
samples = scaler.fit_transform(train)
samples = pd.DataFrame(samples,index=train.index,columns=train.columns) 
samples.fillna(samples.mean(),inplace=True)
train_1 = pd.concat([samples_2.iloc[:,0:2],samples],axis=1)  #with role IDs

X_train_1, X_test_1, y_train, y_test = train_test_split(train_1, target, test_size = 0.2, random_state = 7)
X_train = X_train_1.iloc[:,2:]
X_test = X_test_1.iloc[:,2:]
id_train = X_train_1.iloc[:,0:2]
id_test = X_test_1.iloc[:,0:2]

In [ ]:
pos_X_train = X_train[y_train==1]
pos_X_train_1 = X_train_1[y_train==1] #with role IDs
un_X_train = X_train[y_train==0]
un_X_train_1 = X_train_1[y_train==0] #with role IDs
pos_X_test = X_test[y_test==1]
pos_X_test_1 = X_test_1[y_test==1]
un_X_test = X_test[y_test==0]
un_X_test_1 = X_test_1[y_test==0]

print(pos_X_train.shape)
print(pos_X_test.shape)
print(un_X_train.shape)
print(un_X_test.shape)

# 3. XGBoost

In [ ]:
bst = XGBClassifier(max_depth=6,min_child_weight=2,learning_rate=0.01, n_estimators=100, 
                   objective='binary:logistic',scale_pos_weight=len(un_X_train)//len(pos_X_train)) #sklearn api


bst.fit(X_train, y_train)
xgb_normal = copy.deepcopy(bst)

train_preds = bst.predict_proba(X_train)[:,1]
train_predictions = [1 if value>=0.4 else 0 for value in train_preds]

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions)
train_recall = recall_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
train_auc = roc_auc_score(y_train,train_preds)

print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
print ("Train Precision: %.2f%%" % (train_precision * 100.0))
print ("Train Recall: %.2f%%" % (train_recall * 100.0))
print ("Train f1: %.4f" % (train_f1))
print ("Train AUC: %.4f" % (train_auc))


# make prediction
preds = bst.predict_proba(X_test)[:,1]
predictions = [1 if value>=0.4 else 0 for value in preds] 

test_accuracy = accuracy_score(y_test, predictions)
test_precision = precision_score(y_test, predictions)
test_recall = recall_score(y_test, predictions)
test_f1=f1_score(y_test, predictions)
test_auc=roc_auc_score(y_test,preds)

print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
print ("Test Precision: %.2f%%" % (test_precision * 100.0))
print ("Test Recall: %.2f%%" % (test_recall * 100.0))
print ("Test f1: %.4f" % (test_f1))
print ("Test AUC: %.4f" % (test_auc))

#feature importance
plot_importance(xgb_normal)
plt.show()

# 4. MLP

In [ ]:
#Dataset for training (torch)
class MyDatasets(Dataset):

    def __init__(self,x,y): #numpy to tensor
        self.x = torch.from_numpy(x.values)
        self.y = torch.from_numpy(y.values)

    def __getitem__(self, index):
        return self.x[index],self.y[index] #,self.ids[index,:]

    def __len__(self):
        return len(self.x)

#DataLoader for training (torch)
train_loader = DataLoader(MyDatasets(X_train,y_train), batch_size=1024, shuffle=True)
test_loader = DataLoader(MyDatasets(X_test,y_test), batch_size=1024, shuffle=False)

In [ ]:
#define the net(MLP) architecture
class Net(nn.Module):

    def __init__(self,input_dim,hidden_dim,num_layers,output_dim):

        super(Net,self).__init__()

        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.out_layer = nn.Linear(hidden_dim,output_dim)
        self.hidden_dict = collections.OrderedDict()
        for i in range(num_layers):
            self.hidden_dict['fc'+str(i+1)] = nn.Linear(hidden_dim,hidden_dim)
        self.hidden_layers = nn.Sequential(self.hidden_dict)


    def forward(self,x):
        
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
        out = F.softmax(self.out_layer(x),dim=1)
        return out

In [ ]:
def train(model, device,train_loader, myloss, optimizer, epoch):

    model.train()

    for batch_idx, (train_x, train_y) in enumerate(train_loader):
        train_x = Variable(train_x).to(device)
        train_y = Variable(train_y).to(device)
        optimizer.zero_grad()
        output = model(train_x)
        loss = myloss(output, train_y.long())
        loss.backward()
        optimizer.step()
        if batch_idx%10 == 0:
            print('Train Epoch: {} [{}/{}]\tloss: {:.6f}'.format(
                epoch, batch_idx*len(train_x), len(train_loader.dataset),loss.data.cpu().numpy()))


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #to gpu if available
net = Net(pos_X_train.shape[1],64,5,2)
net = net.to(device)
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net,device_ids=[0,1,2])
print(net)  # net architecture

optimizer = optim.Adam(net.parameters(), lr=0.0001) 
weight = torch.DoubleTensor([len(pos_X_train),len(un_X_train)]).to(device)
loss_func = nn.CrossEntropyLoss(weight=weight)  # the target label is NOT an one-hotted
for epoch in range(50):  #training for a few epochs
    
    train(net,device,train_loader,loss_func,optimizer,epoch)
    

In [ ]:
net_gpu = copy.deepcopy(net)
#net = net.to("cpu")  #single-gpu
net_cpu = net.module.to("cpu")  #multi-gpu
net_normal = copy.deepcopy(net_cpu)

train_preds = net_cpu(torch.from_numpy(X_train.values))[:,1].detach().numpy()
train_predictions = [1 if value>=0.4 else 0 for value in train_preds]

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions)
train_recall = recall_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
train_auc = roc_auc_score(y_train,train_preds)

print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
print ("Train Precision: %.2f%%" % (train_precision * 100.0))
print ("Train Recall: %.2f%%" % (train_recall * 100.0))
print ("Train f1: %.4f" % (train_f1))
print ("Train AUC: %.4f" % (train_auc))


# make prediction
preds = net_cpu(torch.from_numpy(X_test.values))[:,1].detach().numpy()
predictions = [1 if value>=0.4 else 0 for value in preds] 

test_accuracy = accuracy_score(y_test, predictions)
test_precision = precision_score(y_test, predictions)
test_recall = recall_score(y_test, predictions)
test_f1=f1_score(y_test, predictions)
test_auc=roc_auc_score(y_test,preds)

print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
print ("Test Precision: %.2f%%" % (test_precision * 100.0))
print ("Test Recall: %.2f%%" % (test_recall * 100.0))
print ("Test f1: %.4f" % (test_f1))
print ("Test AUC: %.4f" % (test_auc))

# 5. SVM

In [ ]:
svc = SVC(class_weight='balanced')
svc.fit(X_train,y_train)


train_preds = svc.predict_proba(X_train)[:,1]
train_predictions = [1 if value>=0.4 else 0 for value in train_preds]

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions)
train_recall = recall_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
train_auc = roc_auc_score(y_train,train_preds)

print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
print ("Train Precision: %.2f%%" % (train_precision * 100.0))
print ("Train Recall: %.2f%%" % (train_recall * 100.0))
print ("Train f1: %.4f" % (train_f1))
print ("Train AUC: %.4f" % (train_auc))


# make prediction
preds = svc.predict_proba(X_test)[:,1]
predictions = [1 if value>=0.4 else 0 for value in preds] 

test_accuracy = accuracy_score(y_test, predictions)
test_precision = precision_score(y_test, predictions)
test_recall = recall_score(y_test, predictions)
test_f1=f1_score(y_test, predictions)
test_auc=roc_auc_score(y_test,preds)

print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
print ("Test Precision: %.2f%%" % (test_precision * 100.0))
print ("Test Recall: %.2f%%" % (test_recall * 100.0))
print ("Test f1: %.4f" % (test_f1))
print ("Test AUC: %.4f" % (test_auc))

# 6. Random Forest

In [ ]:
rf = RandomForestClassifier(class_weight='balanced',min_samples_split=200,n_estimators=100)
rf.fit(X_train,y_train)

train_preds = rf.predict_proba(X_train)[:,1]
train_predictions = [1 if value>=0.4 else 0 for value in train_preds]

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions)
train_recall = recall_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
train_auc = roc_auc_score(y_train,train_preds)

print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
print ("Train Precision: %.2f%%" % (train_precision * 100.0))
print ("Train Recall: %.2f%%" % (train_recall * 100.0))
print ("Train f1: %.4f" % (train_f1))
print ("Train AUC: %.4f" % (train_auc))


# make prediction
preds = rf.predict_proba(X_test)[:,1]
predictions = [1 if value>=0.4 else 0 for value in preds] 

test_accuracy = accuracy_score(y_test, predictions)
test_precision = precision_score(y_test, predictions)
test_recall = recall_score(y_test, predictions)
test_f1=f1_score(y_test, predictions)
test_auc=roc_auc_score(y_test,preds)

print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
print ("Test Precision: %.2f%%" % (test_precision * 100.0))
print ("Test Recall: %.2f%%" % (test_recall * 100.0))
print ("Test f1: %.4f" % (test_f1))
print ("Test AUC: %.4f" % (test_auc))

# 7. Naive Bayes

In [ ]:
nb = GaussianNB()  #prior calculated by frequency
nb.fit(X_train,y_train)

train_preds = nb.predict_proba(X_train)[:,1]
train_predictions = [1 if value>=0.4 else 0 for value in train_preds]

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions)
train_recall = recall_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
train_auc = roc_auc_score(y_train,train_preds)

print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
print ("Train Precision: %.2f%%" % (train_precision * 100.0))
print ("Train Recall: %.2f%%" % (train_recall * 100.0))
print ("Train f1: %.4f" % (train_f1))
print ("Train AUC: %.4f" % (train_auc))


# make prediction
preds = nb.predict_proba(X_test)[:,1]
predictions = [1 if value>=0.4 else 0 for value in preds] #指定0.4为threshold

test_accuracy = accuracy_score(y_test, predictions)
test_precision = precision_score(y_test, predictions)
test_recall = recall_score(y_test, predictions)
test_f1=f1_score(y_test, predictions)
test_auc=roc_auc_score(y_test,preds)

print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
print ("Test Precision: %.2f%%" % (test_precision * 100.0))
print ("Test Recall: %.2f%%" % (test_recall * 100.0))
print ("Test f1: %.4f" % (test_f1))
print ("Test AUC: %.4f" % (test_auc))

# 8. GAT

In [ ]:
#map node IDs to embeddings
ids = np.unique(np.concatenate([samples_2['roleid_src'],samples_2['roleid_dst']]))
id_dict = {}
for i in range(len(ids)):
    id_dict[ids[i]] = i
id1 = torch.from_numpy(np.array([id_dict[x] for x in samples_2['roleid_src']]))
id2 = torch.from_numpy(np.array([id_dict[x] for x in samples_2['roleid_dst']]))

input_size = 8
features = torch.randn(len(ids),input_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
features = features.to(device)

# consturct the graph using trade amount/frequency
G = dgl.DGLGraph()
#use RMT prediction score(also tried trade amount/trade frequency) as edge weight
G.add_edges(id1,id2,data={'weight':torch.from_numpy(np.array(samples_2['trade_n_times_mean']))})
#add self-loops with weight 1.0
G.add_edges(g.nodes(), g.nodes(),data={'weight':torch.ones_like(g.nodes(),dtype=torch.float32)})
G = G.to(device)
print(G)

In [ ]:
class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        # equation (1)
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        # equation (2)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)
        self.edge_fc1 = nn.Linear(9 , 32, bias=False)
        self.edge_fc2 = nn.Linear(32,1,bias=False)

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        z2 = torch.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        b = self.edge_fc1(edges.data['feat'])
        b = F.elu(b)
        b = self.edge_fc2(b)
        return {'e': F.leaky_relu(a)+F.leaky_relu(b)}

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        # equation (4)
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

    def forward(self, h):
        # equation (1)
        z = self.fc(h)
        self.g.ndata['z'] = z
        # equation (2)
        self.g.apply_edges(self.edge_attention)
        # equation (3) & (4)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')


In [ ]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            return torch.cat(head_outs, dim=1)
        else:
            # merge using average
            return torch.mean(torch.stack(head_outs))


In [ ]:
#model definition
class GAT(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        # Be aware that the input dimension is hidden_dim*num_heads since
        # multiple head outputs are concatenated together. Also, only
        # one attention head in the output layer.
        self.layer2 = MultiHeadGATLayer(g, hidden_dim * num_heads, hidden_dim, 1)
        self.fc = nn.Linear(in_features=hidden_dim,out_features=out_dim)

    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        h = F.sigmoid(self.fc(h))
        return h

In [ ]:
def evaluate(model, graph, features, labels, mask):
    model.eval()
    with torch.no_grad():
        #logits = model(graph, features).cpu()
        logits = model(features).cpu()
        logits = logits.squeeze(dim=-1) 
        logits = logits[mask].cpu()
        labels = labels[mask].cpu()
        indices = logits>=0.5
        #_, indices = torch.max(logits, dim=1)
        #correct = torch.sum(indices == labels)
        return accuracy_score(labels,indices),roc_auc_score(labels,logits),f1_score(labels,indices),precision_score(labels,indices),recall_score(labels,indices)
		#return correct.item() * 1.0 / len(labels) #acc作为评估指标

In [ ]:
#train and test
model = GAT(G,in_dim=n_features,hidden_dim=32,out_dim=1,num_heads=2)
model = model.to(device)
opt = torch.optim.Adam(model.parameters())

for epoch in range(20):
    model.train()
    # forward propagation by using all samples
    logits = model(node_features)
    logits = logits.squeeze(dim=-1) 
    # compute loss
    loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([30]).to(device))(logits[train_mask],node_labels[train_mask].float())
    # compute train/validation accuracy
    train_acc,train_auc,train_f1,train_precision,train_recall=evaluate(model, G, node_features, node_labels, train_mask) 
    valid_acc,valid_auc,valid_f1,valid_precision,valid_recall= evaluate(model, G, node_features, node_labels, valid_mask) 
    print("epoch:"+str(epoch)+" train_acc:"+str(train_acc)+" train_auc:"+str(train_auc)+" train_f1:"+str(train_f1)+" train_precision:"+str(train_precision)+" train_recall:"+str(train_recall))
    print("epoch:"+str(epoch)+" valid_acc:"+str(valid_acc)+" valid_auc:"+str(valid_auc)+" valid_f1:"+str(valid_f1)+" valid_precision:"+str(valid_precision)+" valid_recall:"+str(valid_recall))
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()

# 9. LSTM

In [ ]:
#tansaction logs
samples_1 = pd.read_csv('./data_gnn/samples_1.csv')

In [ ]:
windows_size=16
def get_sample(user_row):

    #user_row = user_row[np.newaxis, ...]
    l = len(user_row)
    x = []
    for row in user_row:
        x.append(np.array(row))
    x = np.array(x)
    #padding the sequence for training
    if l < windows_size:
        return np.pad(x, [[0,windows_size-x.shape[0]], [0, 0]],constant_values=0)
    elif l > windows_size:
        return x[-windows_size:]
    else:
        return x

In [ ]:
users_samples=[]
labels=[]
cut_length=[]
grouped_samples_1=samples_1[['roleid_src','roleid_dst','x','label']].groupby(['roleid_src','roleid_dst']) #这里后面测好了改回samples_2

#organize the transaction sequences
for (roleid_src,roleid_dst),group in grouped_samples_1:
    length=len(group)
    if length==1:
        continue
    else:
        if np.sum(group['label'])>0:
            labels.append(1) 
        else:
            labels.append(0)
        users_samples.append(get_sample(group['x']))
        if length<=windows_size:
            cut_length.append(length)
        else:
            cut_length.append(windows_size)

print(len(labels))
print(np.sum(labels))
print(np.sum(labels)/len(labels))

In [ ]:
#Dataset for training (torch)
class MyDataset(Dataset):
    def __init__(self, x, y,length):
        self.x = x
        self.y = y
        self.length = length

    def __getitem__(self, index):
        
        #return self.x[index], torch.tensor(self.y[index],dtype=torch.float64),self.length[index]
        return self.x[index], self.y[index],self.length[index]

    def __len__(self):
        return len(self.y)

In [ ]:
val_ratio = 0.2
num_val = (int)(val_ratio*len(labels))
print(num_val)

x_train = users_samples[:-num_val]
y_train = labels[:-num_val]
l_train = cut_length[:-num_val]
x_test = users_samples[-num_val:]
y_test = labels[-num_val:]
l_test = cut_length[-num_val:]

#split the train_test dataset
train_dataset = MyDataset(x_train,y_train,l_train)
test_dataset = MyDataset(x_test,y_test,l_test)

#Data Loader for training (torch)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,  shuffle=False)

In [ ]:
class LSTM_1(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=32, output_size=2):
        super().__init__()

        #LSTM+lienar
        self.linear_1 = nn.Linear(input_size, hidden_layer_size)
        self.dropout = nn.Dropout(p = 0.5)
        self.lstm = nn.LSTM(hidden_layer_size, hidden_layer_size)
        self.linear_2 = nn.Linear(hidden_layer_size, output_size)



    def forward(self, input_seq):

        x = self.linear_1(input_seq[0])
        x = F.relu(x)
        x = self.dropout(x)
        data_packed_input = pack_padded_sequence(input=x, lengths=input_seq[1], batch_first=True,enforce_sorted=False)
        _, (h_t,_) = self.lstm(data_packed_input)
        predictions = self.linear_2(h_t.squeeze(0)) #undirected ltsm
        predictions = F.softmax(predictions,dim=-1)
        return predictions #the last one

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 64
num_epochs = 20
embedding_size = 64
hidden_size = 64
log_interval = 100
save_model = False

In [ ]:
# for training
def train_1(log_interval, model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target_0, lengths) in enumerate(train_loader):
        optimizer.zero_grad()
        data, target_0,lengths = data.to(device), target_0.to(device), lengths.to(device)
        output = model((data,lengths))
        

        target = target_0.unsqueeze(1)
        target = torch.LongTensor(target)
        target = torch.zeros(len(lengths),2).scatter_(1,target,1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader.dataset), loss.item()))



In [ ]:
# for testing
def test_1(model, device, data, target_0, lengths, criterion, train_string): #valid or test
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        data, target_0,lengths = data.to(device), target_0.to(device), lengths.to(device)
        output = model((data,lengths)) 

        target = target_0.unsqueeze(1)
        target = torch.LongTensor(target)
        target = torch.zeros(len(lengths),2).scatter_(1,target,1)

        train_loss = criterion(output, target).item()
        train_preds = F.softmax(output,1)[:,1].detach().numpy() #logits
        train_predictions = output.argmax(dim=1, keepdim=True).detach().numpy()
        y_train = target_0.detach().numpy()
        train_accuracy = accuracy_score(y_train, train_predictions)
        train_precision = precision_score(y_train, train_predictions)
        train_recall = recall_score(y_train, train_predictions)
        train_f1 = f1_score(y_train, train_predictions)
        train_auc = roc_auc_score(y_train,train_preds)
        
        print(train_string +' set: Average loss: {:.4f}'.format(train_loss))
        print(train_string + " Accuary: %.2f%%" % (train_accuracy * 100.0))
        print(train_string +" Precision: %.2f%%" % (train_precision * 100.0))
        print(train_string +" Recall: %.2f%%" % (train_recall * 100.0))
        print(train_string +" f1: %.4f" % (train_f1))
        print(train_string +" AUC: %.4f" % (train_auc))
 
 

In [ ]:
#define the model and loss/optimizer
model_1 = LSTM_1(input_size = len(users_samples[0][0])).to(device)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight = torch.FloatTensor([1,100]))
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = torch.optim.Adam(model_1.parameters())
#,lr=0.001,betas=(0.9, 0.999),eps=1e-08,weight_decay=0,amsgrad=False)

#training
for epoch in range(1, num_epochs + 1):
    print(epoch)
    train_1(log_interval, model_1, device, train_loader, criterion, optimizer, epoch)
    test_1(model_1, device, torch.tensor(x_train), torch.tensor(y_train), torch.tensor(l_train), criterion,"Train")
    test_1(model_1, device, torch.tensor(x_test), torch.tensor(y_test), torch.tensor(l_test), criterion,"Test")

torch.save(model_1.state_dict(), "test_lstm.pt")


In [ ]:
net_gpu = copy.deepcopy(model_1)
#net = net.to("cpu")  #single-gpu
net_cpu = model_1.module.to("cpu")  #multi-gpu
net_normal = copy.deepcopy(net_cpu)

train_preds = net_cpu(torch.from_numpy(x_train))[:,1].detach().numpy()
train_predictions = [1 if value>=0.4 else 0 for value in train_preds]

train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions)
train_recall = recall_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
train_auc = roc_auc_score(y_train,train_preds)

print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
print ("Train Precision: %.2f%%" % (train_precision * 100.0))
print ("Train Recall: %.2f%%" % (train_recall * 100.0))
print ("Train f1: %.4f" % (train_f1))
print ("Train AUC: %.4f" % (train_auc))


# make prediction
preds = net_cpu(torch.from_numpy(x_test))[:,1].detach().numpy()
predictions = [1 if value>=0.4 else 0 for value in preds] 

test_accuracy = accuracy_score(y_test, predictions)
test_precision = precision_score(y_test, predictions)
test_recall = recall_score(y_test, predictions)
test_f1=f1_score(y_test, predictions)
test_auc=roc_auc_score(y_test,preds)

print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
print ("Test Precision: %.2f%%" % (test_precision * 100.0))
print ("Test Recall: %.2f%%" % (test_recall * 100.0))
print ("Test f1: %.4f" % (test_f1))
print ("Test AUC: %.4f" % (test_auc))